***LC_net***

In [7]:
import torch
import torch.nn as nn
from accelerate import Accelerator
from torch.optim.optimizer import Optimizer, required
from torchvision.models import resnet50
from transformers import ViTModel, ViTConfig
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, RandomResizedCrop, v2, Resize
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from accelerate import notebook_launcher
import os
import timm
import time

os.environ["CUDA_VISIBLE_DEVICES"] = '0'


class lcnet_ft(nn.Module):
    def __init__(self, num_classes=10, embed_dim=1280):
        super(lcnet_ft, self).__init__()

        self.embed_dim = embed_dim
        
        # Load and freeze lcnet
        self.encoder = timm.create_model('timm/lcnet_075.ra2_in1k', pretrained=True, num_classes=num_classes)
        for param in self.encoder.parameters():
            param.requires_grad = False

        # unfreeze the last layer
        # for param in self.encoder.classifier.parameters():
        #     param.requires_grad = True

        self.encoder.classifier = nn.Identity()


        self.classifier = nn.Sequential(
            nn.Linear(self.embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        # # Extract features from ResNet50 and ViT
        logits = self.encoder(x)  # Assuming [batch_size, embed_dim]

        logits = self.classifier(logits)
        
        return logits


# Assuming CIFAR10 - adapt transforms for your dataset
transform = Compose([
    Resize((224,224)),
    RandomHorizontalFlip(),
    # v2.ColorJitter(brightness=(0.5,1.5),contrast=(1),saturation=(0.5,1.5),hue=(-0.1,0.1)),
    ToTensor(),
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
])

# Dataset and DataLoader setup
class FTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def train_model(train_loader, test_loader):
    accelerator = Accelerator(mixed_precision="fp16")
    model = lcnet_ft()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()

    model, optimizer, train_loader, test_loader = accelerator.prepare(model, optimizer, train_loader, test_loader)

    # time start
    
    start = time.time()
    for epoch in range(10):  # Adjust the number of epochs if needed
        model.train()
        total_loss = 0
        for x, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            accelerator.backward(loss)
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")
    # time end
    end = time.time()
    print(f"Training time: {end - start}")

    # Optional test phase at the end of training

    model.eval()
    test_loss = 0
    with torch.no_grad():
        # time start
        start = time.time()
        for x, labels in test_loader:
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
        # time end
        end = time.time()
        print(f"Inference time: {(end - start)/len(test_loader)}")
    avg_test_loss = test_loss / len(test_loader)
    print(f"Test Loss: {avg_test_loss:.4f}")

    # Save the final model
    model_path = "/home/chen/EECE570/new_models/single_model/model_lcnet_ft_epoch_10.pt"
    unencapsulated_model = accelerator.unwrap_model(model)
    accelerator.save(unencapsulated_model.state_dict(), model_path)


def main():
    root_dir = './data'
    batch_size = 128
    transform = Compose([
        Resize((224,224)),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
    ])
    
    # Load full dataset
    full_dataset = CIFAR10(root=root_dir, train=True, download=True, transform=None)
    images = [image for image, _ in full_dataset]
    labels = [label for _, label in full_dataset]

    # Stratified train-test split
    train_images, test_images, train_labels, test_labels = train_test_split(
        images, labels, test_size=0.2, stratify=labels, random_state=42
    )

    # Creating datasets
    train_dataset = FTDataset(train_images, train_labels, transform=transform)
    test_dataset = FTDataset(test_images, test_labels, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    # Call train_model or any other training function here
    train_model(train_loader, test_loader)

notebook_launcher(main, num_processes=1)

Launching training on one GPU.
Files already downloaded and verified


Epoch 1: 100%|██████████| 313/313 [00:08<00:00, 36.98it/s]


Epoch 1, Loss: 0.9201


Epoch 2: 100%|██████████| 313/313 [00:08<00:00, 38.13it/s]


Epoch 2, Loss: 0.5739


...

Epoch 9: 100%|██████████| 313/313 [00:08<00:00, 37.22it/s]


Epoch 9, Loss: 0.4232


Epoch 10: 100%|██████████| 313/313 [00:08<00:00, 37.30it/s]

Epoch 10, Loss: 0.4025
Training time: 83.16917586326599


Inference time: 0.030500070958197872
Test Loss: 0.5143


In [2]:
# testing on testset
model = lcnet_ft()
model.load_state_dict(torch.load("/home/chen/EECE570/new_models/single_model/model_lcnet_ft_epoch_10.pt"))
model.cuda()

full_dataset = CIFAR10(root='./data', train=False, download=True, transform=None)
images = [image for image, _ in full_dataset]
labels = [label for _, label in full_dataset]


testset = FTDataset(images, labels, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)


model.eval()
total = 0
correct = 0
false = 0
false_positive = 0
false_negative = 0
with torch.no_grad():
    for x, labels in tqdm(testloader):
        x, labels = x.cuda(), labels.cuda()
        outputs = model(x)
        predictions = torch.argmax(outputs, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
        false_positive += ((predictions == 1) & (labels == 0)).sum().item()
        false_negative += ((predictions == 0) & (labels == 1)).sum().item()


# output accuracy, recell, precision, f1 score
accuracy = correct / total
recall = correct / (correct + false_negative)
precision = correct / (correct + false_positive)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))
print("F1 Score: {:.4f}".format(f1_score))


Files already downloaded and verified


100%|██████████| 157/157 [00:02<00:00, 57.97it/s]

Accuracy: 0.8177
Recall: 0.9982
Precision: 0.9994
F1 Score: 0.9988


**timm/mnasnet_small.lamb_in1k**

In [8]:

class mnasnet_ft(nn.Module):
    def __init__(self, num_classes=10, embed_dim=1280):
        super(mnasnet_ft, self).__init__()

        self.embed_dim = embed_dim
        
        # Load and freeze lcnet
        self.encoder = timm.create_model('timm/mnasnet_small.lamb_in1k', pretrained=True, num_classes=num_classes)
        for param in self.encoder.parameters():
            param.requires_grad = False

        # unfreeze the last layer
        # for param in self.encoder.classifier.parameters():
        #     param.requires_grad = True

        self.encoder.classifier = nn.Identity()


        self.classifier = nn.Sequential(
            nn.Linear(self.embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        # # Extract features from ResNet50 and ViT
        logits = self.encoder(x)  # Assuming [batch_size, embed_dim]

        logits = self.classifier(logits)
        
        return logits


# Assuming CIFAR10 - adapt transforms for your dataset
transform = Compose([
    Resize((224,224)),
    RandomHorizontalFlip(),
    # v2.ColorJitter(brightness=(0.5,1.5),contrast=(1),saturation=(0.5,1.5),hue=(-0.1,0.1)),
    ToTensor(),
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
])

# Dataset and DataLoader setup
class FTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def train_model(train_loader, test_loader):
    accelerator = Accelerator(mixed_precision="fp16")
    model = mnasnet_ft()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()

    model, optimizer, train_loader, test_loader = accelerator.prepare(model, optimizer, train_loader, test_loader)

    # time start
    
    start = time.time()
    for epoch in range(10):  # Adjust the number of epochs if needed
        model.train()
        total_loss = 0
        for x, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            accelerator.backward(loss)
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")
    # time end
    end = time.time()
    print(f"Training time: {end - start}")

    # Optional test phase at the end of training

    model.eval()
    test_loss = 0
    with torch.no_grad():
        # time start
        start = time.time()
        for x, labels in test_loader:
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
        # time end
        end = time.time()
        print(f"Inference time: {(end - start)/len(test_loader)}")
    avg_test_loss = test_loss / len(test_loader)
    print(f"Test Loss: {avg_test_loss:.4f}")

    # Save the final model
    model_path = "/home/chen/EECE570/new_models/single_model/model_mnasnet_ft_epoch_10.pt"
    unencapsulated_model = accelerator.unwrap_model(model)
    accelerator.save(unencapsulated_model.state_dict(), model_path)


def main():
    root_dir = './data'
    batch_size = 128
    transform = Compose([
        Resize((224,224)),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
    ])
    
    # Load full dataset
    full_dataset = CIFAR10(root=root_dir, train=True, download=True, transform=None)
    images = [image for image, _ in full_dataset]
    labels = [label for _, label in full_dataset]

    # Stratified train-test split
    train_images, test_images, train_labels, test_labels = train_test_split(
        images, labels, test_size=0.2, stratify=labels, random_state=42
    )

    # Creating datasets
    train_dataset = FTDataset(train_images, train_labels, transform=transform)
    test_dataset = FTDataset(test_images, test_labels, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    # Call train_model or any other training function here
    train_model(train_loader, test_loader)

notebook_launcher(main, num_processes=1)

Launching training on one GPU.
Files already downloaded and verified


Epoch 1: 100%|██████████| 313/313 [00:08<00:00, 36.33it/s]


Epoch 1, Loss: 0.8276


Epoch 2: 100%|██████████| 313/313 [00:08<00:00, 36.13it/s]


Epoch 2, Loss: 0.5269


...

Epoch 9: 100%|██████████| 313/313 [00:08<00:00, 36.13it/s]


Epoch 9, Loss: 0.3374


Epoch 10: 100%|██████████| 313/313 [00:08<00:00, 36.37it/s]

Epoch 10, Loss: 0.3121
Training time: 86.26169657707214


Inference time: 0.030616383009319063
Test Loss: 0.4391


In [9]:
# testing on testset
model = mnasnet_ft()
model.load_state_dict(torch.load("/home/chen/EECE570/new_models/single_model/model_mnasnet_ft_epoch_10.pt"))
model.cuda()

full_dataset = CIFAR10(root='./data', train=False, download=True, transform=None)
images = [image for image, _ in full_dataset]
labels = [label for _, label in full_dataset]


testset = FTDataset(images, labels, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)



model.eval()
total = 0
correct = 0
false = 0
false_positive = 0
false_negative = 0
with torch.no_grad():
    for x, labels in tqdm(testloader):
        x, labels = x.cuda(), labels.cuda()
        outputs = model(x)
        predictions = torch.argmax(outputs, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
        false_positive += ((predictions == 1) & (labels == 0)).sum().item()
        false_negative += ((predictions == 0) & (labels == 1)).sum().item()


# output accuracy, recell, precision, f1 score
accuracy = correct / total
recall = correct / (correct + false_negative)
precision = correct / (correct + false_positive)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))
print("F1 Score: {:.4f}".format(f1_score))


Files already downloaded and verified


100%|██████████| 157/157 [00:02<00:00, 61.94it/s]

Accuracy: 0.8484
Recall: 0.9985
Precision: 0.9988
F1 Score: 0.9986


***timm/repghostnet_050.in1k***

In [10]:
class repghostnet_ft(nn.Module):
    def __init__(self, num_classes=10, embed_dim=1280):
        super(repghostnet_ft, self).__init__()

        self.embed_dim = embed_dim
        
        # Load and freeze lcnet
        self.encoder = timm.create_model('timm/repghostnet_050.in1k', pretrained=True, num_classes=num_classes)
        for param in self.encoder.parameters():
            param.requires_grad = False

        # unfreeze the last layer
        # for param in self.encoder.classifier.parameters():
        #     param.requires_grad = True

        self.encoder.classifier = nn.Identity()


        self.classifier = nn.Sequential(
            nn.Linear(self.embed_dim, embed_dim),
            nn.ReLU(),
            nn.Linear(embed_dim, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        # # Extract features from ResNet50 and ViT
        logits = self.encoder(x)  # Assuming [batch_size, embed_dim]

        logits = self.classifier(logits)
        
        return logits


# Assuming CIFAR10 - adapt transforms for your dataset
transform = Compose([
    Resize((224,224)),
    RandomHorizontalFlip(),
    # v2.ColorJitter(brightness=(0.5,1.5),contrast=(1),saturation=(0.5,1.5),hue=(-0.1,0.1)),
    ToTensor(),
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
])

# Dataset and DataLoader setup
class FTDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

def train_model(train_loader, test_loader):
    accelerator = Accelerator(mixed_precision="fp16")
    model = repghostnet_ft()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.CrossEntropyLoss()

    model, optimizer, train_loader, test_loader = accelerator.prepare(model, optimizer, train_loader, test_loader)

    # time start
    
    start = time.time()
    for epoch in range(10):  # Adjust the number of epochs if needed
        model.train()
        total_loss = 0
        for x, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            accelerator.backward(loss)
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")
    # time end
    end = time.time()
    print(f"Training time: {end - start}")

    # Optional test phase at the end of training

    model.eval()
    test_loss = 0
    with torch.no_grad():
        # time start
        start = time.time()
        for x, labels in test_loader:
            outputs = model(x)
            loss = loss_fn(outputs, labels)
            test_loss += loss.item()
        # time end
        end = time.time()
        print(f"Inference time: {(end - start)/len(test_loader)}")
    avg_test_loss = test_loss / len(test_loader)
    print(f"Test Loss: {avg_test_loss:.4f}")

    # Save the final model
    model_path = "/home/chen/EECE570/new_models/single_model/model_repghostnet_ft_epoch_10.pt"
    unencapsulated_model = accelerator.unwrap_model(model)
    accelerator.save(unencapsulated_model.state_dict(), model_path)


def main():
    root_dir = './data'
    batch_size = 128
    transform = Compose([
        Resize((224,224)),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]),
    ])
    
    # Load full dataset
    full_dataset = CIFAR10(root=root_dir, train=True, download=True, transform=None)
    images = [image for image, _ in full_dataset]
    labels = [label for _, label in full_dataset]

    # Stratified train-test split
    train_images, test_images, train_labels, test_labels = train_test_split(
        images, labels, test_size=0.2, stratify=labels, random_state=42
    )

    # Creating datasets
    train_dataset = FTDataset(train_images, train_labels, transform=transform)
    test_dataset = FTDataset(test_images, test_labels, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    # Call train_model or any other training function here
    train_model(train_loader, test_loader)

notebook_launcher(main, num_processes=1)

Launching training on one GPU.
Files already downloaded and verified


Epoch 1: 100%|██████████| 313/313 [00:08<00:00, 35.11it/s]


Epoch 1, Loss: 1.0734


Epoch 2: 100%|██████████| 313/313 [00:08<00:00, 34.87it/s]


Epoch 2, Loss: 0.7103


...

Epoch 9: 100%|██████████| 313/313 [00:08<00:00, 34.99it/s]


Epoch 9, Loss: 0.6022


Epoch 10: 100%|██████████| 313/313 [00:08<00:00, 34.95it/s]

Epoch 10, Loss: 0.5884
Training time: 89.75499534606934


Inference time: 0.030260659471342835
Test Loss: 0.5988


In [11]:
# testing on testset
model = repghostnet_ft()
model.load_state_dict(torch.load("/home/chen/EECE570/new_models/single_model/model_repghostnet_ft_epoch_10.pt"))
model.cuda()

full_dataset = CIFAR10(root='./data', train=False, download=True, transform=None)
images = [image for image, _ in full_dataset]
labels = [label for _, label in full_dataset]


testset = FTDataset(images, labels, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)



model.eval()
total = 0
correct = 0
false = 0
false_positive = 0
false_negative = 0
with torch.no_grad():
    for x, labels in tqdm(testloader):
        x, labels = x.cuda(), labels.cuda()
        outputs = model(x)
        predictions = torch.argmax(outputs, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()
        false_positive += ((predictions == 1) & (labels == 0)).sum().item()
        false_negative += ((predictions == 0) & (labels == 1)).sum().item()


# output accuracy, recell, precision, f1 score
accuracy = correct / total
recall = correct / (correct + false_negative)
precision = correct / (correct + false_positive)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy: {:.4f}".format(accuracy))
print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))
print("F1 Score: {:.4f}".format(f1_score))


Files already downloaded and verified


100%|██████████| 157/157 [00:02<00:00, 65.44it/s]

Accuracy: 0.7824
Recall: 0.9981
Precision: 0.9962
F1 Score: 0.9971
